In [1]:
import pandas as pd
import altair as alt
import numpy as np
alt.data_transformers.disable_max_rows() # Disable 5_000 rows limit
findings_data = pd.read_csv("https://code4rena.com/community-resources/findings.csv")
contests_data = pd.read_csv("https://code4rena.com/community-resources/contests.csv")

In [2]:
df = findings_data.copy()
df['end_time'] = pd.to_datetime(findings_data['contest'].map(contests_data.drop_duplicates().set_index('contestid')['end_time']).str[:25]).dt.normalize()
tmp = df.groupby(['handle']).agg({'end_time' : np.min}).reset_index().groupby('end_time').count().reset_index()
df = pd.merge(df, tmp, how='left', on=['end_time']).rename(columns={'handle_x': 'handle', 'handle_y': 'new_wardens'})
df['new_wardens'].fillna(0, inplace=True)

/tmp/ipykernel_3052/1990220689.py:3: FutureWarning: The provided callable <function min at 0x7388ec30c860> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  tmp = df.groupby(['handle']).agg({'end_time' : np.min}).reset_index().groupby('end_time').count().reset_index()


In [3]:
N_PERIODS = 6

`N_PERIODS`: number of months for the total wardens trend projection

In [4]:
df["end_time"] = pd.to_datetime(df["end_time"])
plt_data = pd.DataFrame()

# Active wardens at the time
plt_data["active_wardens"] = df.groupby('end_time')['handle'].nunique()

# Inactive wardens (only one or less contest participation total)
one_timers = df.groupby(
    ['handle', 'end_time']
).size().groupby(level=0).size().to_frame()[lambda x: x.iloc[:, [0]] <= 1].dropna().reset_index()["handle"]
plt_data['inactive_wardens'] = df[df["handle"].isin(one_timers)].groupby('end_time')['handle'].nunique().shift(1, fill_value=0)
plt_data['inactive_wardens'].fillna(0., inplace=True)

# New wardens at the time
plt_data['new_wardens'] = df.groupby('end_time')['new_wardens'].mean()

# Non-participating wardens (registered wardens not seen at the time)
plt_data['total_inactive_wardens'] = plt_data['inactive_wardens'].cumsum()
plt_data['total_wardens'] = plt_data['new_wardens'].cumsum()
plt_data['non_participating_wardens'] = plt_data['total_wardens'] - plt_data['active_wardens'] - plt_data['total_inactive_wardens']
plt_data['predicted'] = False

# Generate prediction data based on mean difference
c1, c2, c3 = (
    round(plt_data.groupby(pd.Grouper(freq="M"))['active_wardens'].mean().diff().mean()),
    round(plt_data.groupby(pd.Grouper(freq="M"))['total_inactive_wardens'].mean().diff().mean()),
    round(plt_data.groupby(pd.Grouper(freq="M"))['non_participating_wardens'].mean().diff().mean())
)
last_date = plt_data.index[-1]

l = []
for (i, d) in enumerate(pd.date_range(start=last_date, periods=N_PERIODS, freq='MS')):
    if i == 0:
        l.append({
            'end_time': d, 
            'predicted': True, 
            'active_wardens': plt_data['active_wardens'].iloc[len(plt_data)-i-1] + c1,
            'total_inactive_wardens': plt_data['total_inactive_wardens'].iloc[len(plt_data)-i-1] + c2,
            'non_participating_wardens': plt_data['non_participating_wardens'].iloc[len(plt_data)-i-1] + c3
        })
    else:
        l.append({
            'end_time': d,
            'predicted': True,
            'active_wardens': l[i-1]['active_wardens'] + c1,
            'total_inactive_wardens': l[i-1]['total_inactive_wardens'] + c2,
            'non_participating_wardens': l[i-1]['non_participating_wardens'] + c3
        })
    l[-1]['total_wardens'] = l[-1]['active_wardens'] + l[-1]['total_inactive_wardens'] + l[-1]['non_participating_wardens']

tmp = pd.DataFrame(l)
tmp['end_time'] = pd.to_datetime(tmp['end_time'])
tmp.set_index('end_time', inplace=True)

plt_data = pd.concat([plt_data, tmp])

# Show NaN values (should only be for predicted rows)
plt_data[plt_data.isnull().any(axis=1)]

,active_wardens,inactive_wardens,new_wardens,total_inactive_wardens,total_wardens,non_participating_wardens,predicted
end_time,,,,,,,
2023-12-01,124,NaN,NaN,649.0,1999.0,1226.0,True
2024-01-01,128,NaN,NaN,667.0,2057.0,1262.0,True
2024-02-01,132,NaN,NaN,685.0,2115.0,1298.0,True
2024-03-01,136,NaN,NaN,703.0,2173.0,1334.0,True
2024-04-01,140,NaN,NaN,721.0,2231.0,1370.0,True
2024-05-01,144,NaN,NaN,739.0,2289.0,1406.0,True


In [5]:
alt.Chart(
    plt_data.reset_index(),
    width=750,
    height=500,
    title="Total number of wardens (at least one registered finding)"
).transform_fold(
    ['non_participating_wardens', 'total_inactive_wardens', 'active_wardens'],
    as_=['column', 'value']
).transform_calculate(
    percent='datum.value / datum.total_wardens'
).mark_bar(
    size=15
).encode(
    x=alt.X('yearmonth(end_time):T', axis=alt.Axis(title="", grid=False)),
    y=alt.Y('mean(value):Q', axis=alt.Axis(title="")),
    tooltip=['yearmonth(end_time):T', 'mean(value):Q', alt.Tooltip('mean(percent):Q', format=".0%"), 'column:N', 'predicted:O'],
    color=alt.Color('column:N', title=""),
    # Had to see debug info to figure out the right datum -> https://stackoverflow.com/a/65851500
    opacity=alt.condition(f'toDate(datum.yearmonth_end_time) < toDate("{last_date}")', alt.value(1), alt.value(0.5))
).interactive()

alt.Chart(...)

# Analysis

**Tip**
> You can zoom and interact with the chart for better visualization.
> You can also set the `N_PERIODS` variable above to increase the prediction's period.

**Legend definitions**
- *Active* wardens: wardens that have participated in a contest for the month.
- *Non-participating* wardens: wardens that have participated in previous contests but were not active for the month.
- *Inactive* wardens: wardens that have only participated once.

The exponential growth seems to have slowed down starting in 2023. It looks like we're heading towards a steady increase of newcomers that may balance out the growing number of *inactive* wardens.

Indeed, almost $\frac{1}{3}$ of participating wardens only participate in **one** contest (I'm guilty of this as well :eyes:). The *active* community only represent about **6%** of the total warden population at any time, that's a loss for the sponsors that don't see as many eyes on their code as they can hope for.

In [6]:
# Number of findings per contest and categories

In [7]:
# Reward per contest